In [ ]:
!pip install -q streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.0 MB/s eta 0:00:00


In [ ]:
%%writefile train.py

import torch
import numpy as np
import json
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

with open('intents.json', 'r') as f:
    intents = json.load(f)


tags = []
for intent in intents['intents']:
    tags.append(intent['tag'])
tags = sorted(set(tags))


input_ids = []
attention_masks = []
labels = []


for intent in intents['intents']:
    for pattern in intent['patterns']:
        encoded = tokenizer(
            pattern,
            add_special_tokens=True,
            max_length=20,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )


        input_ids.append(encoded['input_ids'][0])
        attention_masks.append(encoded['attention_mask'][0])

        label_ids = tags.index(intent['tag'])
        labels.append(label_ids)


input_ids = torch.stack(input_ids)
attention_masks = torch.stack(attention_masks)
labels = torch.tensor(labels)


class ChatDataset(Dataset):
    def __init__(self, encodings, mask, labels):

        self.encodings = encodings
        self.mask = mask
        self.labels = labels

    def __getitem__(self, idx):

        return {
            'input_ids': self.encodings[idx],
            'attention_mask': self.mask[idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)


dataset = ChatDataset(input_ids, attention_masks, labels)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)


class Bert_Arch(nn.Module):
    def __init__(self, output_dim):
        super(Bert_Arch, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')


        # for param in self.bert.parameters():
        #     param.requires_grad = False

        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, output_dim)

    def forward(self, sent_id, mask):
        output = self.bert(sent_id, attention_mask=mask)
        cls_vector = output.pooler_output
        x = self.fc(self.dropout(cls_vector))
        return x

output_dim = len(tags)
model = Bert_Arch(output_dim)
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
cross_entropy = nn.CrossEntropyLoss()



epochs = 50

for epoch in range(epochs):
    total_loss = 0

    for batch in train_loader:
        sent_id = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        model.zero_grad()


        preds = model(sent_id, mask)

        loss = cross_entropy(preds, labels)
        total_loss = total_loss + loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()


    avg_loss = total_loss / len(train_loader)

    if (epoch+1) % 20 == 0:
        print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f}")


output_data = {
    "model_state": model.state_dict(),
    "output_dim": output_dim,
    "tags": tags,
    "vocab_size": len(tokenizer),
    "embed_dim": 768,
    "hidden_size": 768,
    "max_len": 20
}






torch.save(output_data, "bert_data.pth")





Writing train.py


In [ ]:
!python train.py

tokenizer_config.json: 100% 48.0/48.0 [00:00<00:00, 193kB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 830kB/s]
tokenizer.json: 100% 466k/466k [00:00<00:00, 775kB/s]
config.json: 100% 570/570 [00:00<00:00, 1.90MB/s]
model.safetensors: 100% 440M/440M [00:03<00:00, 115MB/s]
Loading weights: 100% 199/199 [00:00<00:00, 1104.43it/s, Materializing param=pooler.dense.weight]
BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.bias                       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can b

In [ ]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import json
import random


st.set_page_config(page_title="AI Chatbot", page_icon="🤖")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class Bert_Arch(nn.Module):
    def __init__(self, output_dim):
        super(Bert_Arch, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, output_dim)

    def forward(self, input_ids, attention_mask):

        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_vector = output.pooler_output
        x = self.fc(self.dropout(cls_vector))
        return x


@st.cache_resource
def load_brain():
    with open('intents.json', 'r') as f:
        intents = json.load(f)

    if torch.cuda.is_available():
        data = torch.load('bert_data.pth')
    else:
        data = torch.load('bert_data.pth', map_location=torch.device('cpu'))

    model_state = data['model_state']
    output_dim = data['output_dim']
    tags = data['tags']
    max_len = data['max_len']

    model = Bert_Arch(output_dim)
    model.load_state_dict(model_state)
    model.to(device)
    model.eval()

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    return tokenizer, model, intents, tags, max_len


if 'logged_in' not in st.session_state:
    st.session_state.logged_in = False

def login():
    st.title(" Login to AI Bot")
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")

    if st.button("Login"):
        if username == "admin" and password == "123":
            st.session_state.logged_in = True
            st.rerun()
        else:
            st.error("Incorrect Username or Password")

def logout():
    st.session_state.logged_in = False
    st.rerun()


if not st.session_state.logged_in:
    login()
else:



    with st.sidebar:
        st.write("Welcome, User!")
        if st.button("Logout"):
            logout()


    tokenizer, model, intents, tags, max_len = load_brain()


    st.title("🤖 General Purpose AI Chatbot")
    st.write("Ask about Pakistan, Jokes, Weather, etc.")


    user_input = st.text_input("You:")

    if user_input and st.button("Send"):

        tokens = tokenizer(
            user_input,
            padding='max_length',
            truncation=True,
            max_length=max_len,
            return_tensors='pt'
        )

        input_ids = tokens['input_ids'].to(device)
        attention_mask = tokens['attention_mask'].to(device)


        with torch.no_grad():
            outputs = model(input_ids, attention_mask)


        prob = torch.softmax(outputs, dim=1)
        conf, pred = torch.max(prob, dim=1)
        tag = tags[pred.item()]

        confidence_score = conf.item()

        if confidence_score > 0.50:
            found = False
            for intent in intents['intents']:
                if intent['tag'] == tag:
                    response = random.choice(intent['responses'])
                    st.success(f"Sam: {response}")
                    found = True
                    break
            if not found:
                st.error("Sam: Error finding intent in DB.")
        else:
            st.error(f"Sam: I do not understand... (Confidence: {confidence_score:.2f})")

Writing app.py


In [ ]:
!streamlit run app.py --server.port 8501 --server.enableCORS false &>/content/logs.txt &


In [ ]:
!ps aux | grep streamlit


root        2262 59.8  0.5 236300 73752 ?        Sl   10:16   0:02 /usr/bin/python3 /usr/local/bin/streamlit run app.py --server.port 8501 --server.enableCORS false
root        2288  0.0  0.0   7376  3532 ?        S    10:16   0:00 /bin/bash -c ps aux | grep streamlit
root        2290  0.0  0.0   6484  2344 ?        S    10:16   0:00 grep streamlit


In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64


--2026-02-03 10:16:16--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2026.1.2/cloudflared-linux-amd64 [following]
--2026-02-03 10:16:17--  https://github.com/cloudflare/cloudflared/releases/download/2026.1.2/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/31ade8d6-2fcd-4925-9218-5534d27a01dc?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-02-03T10%3A50%3A55Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-02-

In [ ]:
!chmod +x cloudflared-linux-amd64


In [ ]:
!ls


app.py	       cloudflared-linux-amd64	logs.txt     train.py
bert_data.pth  intents.json		sample_data


In [ ]:
!./cloudflared-linux-amd64 tunnel --url http://localhost:8501


2026-02-03T10:16:31Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-02-03T10:16:31Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-02-03T10:16:37Z INF +--------------------------------------------------------------------------------------------+
2026-02-03T10:16:37Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-02-03T10:16:37Z INF |  https://spots-journalist-warner-variety.trycloudflare